# Homework 10: `SQL`

## Due Date:  Thursday, November 16th at 11:59 PM

You will create a database of the NASA polynomial coefficients for each specie.

**Please turn in your database with your `Jupyter` notebook!**

# Question 1: Convert XML to a SQL database

Create two tables named `LOW` and `HIGH`, each corresponding to data given for the low and high temperature range.
Each should have the following column names:

- `SPECIES_NAME`
- `TLOW`
- `THIGH`
- `COEFF_1`
- `COEFF_2`
- `COEFF_3`
- `COEFF_4`
- `COEFF_5`
- `COEFF_6`
- `COEFF_7`

Populate the tables using the XML file you created in last assignment. If you did not complete the last assignment, you may also use the `example_thermo.xml` file.

`TLOW` should refer to the temperature at the low range and `THIGH` should refer to the temperature at the high range.  For example, in the `LOW` table, $H$ would have `TLOW` at $200$ and `THIGH` at $1000$ and in the `HIGH` table, $H$ would have `TLOW` at $1000$ and `THIGH` at $3500$.

For both tables, `COEFF_1` through `COEFF_7` should be populated with the corresponding coefficients for the low temperature data and high temperature data.

In [1]:
from IPython.display import display
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

In [2]:
import sqlite3
import numpy as np
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

db = sqlite3.connect('HW_10_final_database.sqlite')
cursor = db.cursor()
cursor.execute("DROP TABLE IF EXISTS LOW")
cursor.execute("DROP TABLE IF EXISTS HIGH")
cursor.execute("PRAGMA foreign_keys=1")

cursor.execute('''CREATE TABLE LOW (SPECIES_NAME TEXT, 
TLOW INTEGER, 
THIGH INTEGER, 
COEFF_1 FLOAT, 
COEFF_2 FLOAT, 
COEFF_3 FLOAT, 
COEFF_4 FLOAT, 
COEFF_5 FLOAT, 
COEFF_6 FLOAT, 
COEFF_7 FLOAT)''')

db.commit() # Commit changes to the database

cursor.execute('''CREATE TABLE HIGH (SPECIES_NAME TEXT, 
TLOW INTEGER, 
THIGH INTEGER, 
COEFF_1 FLOAT, 
COEFF_2 FLOAT, 
COEFF_3 FLOAT, 
COEFF_4 FLOAT, 
COEFF_5 FLOAT, 
COEFF_6 FLOAT, 
COEFF_7 FLOAT)''')

db.commit()


In [3]:
import xml.etree.ElementTree as ET
#use the template file
tree = ET.parse('example_thermo.xml')
root = tree.getroot()

species = root.find('speciesData').findall('species')

for specie in species:
    specie_name = specie.get('name')
    #print(specie_name)
    NASA = specie.find('thermo').findall('NASA')
    #LOW
    tmax = NASA[0].get('Tmax')
    tmin = NASA[0].get('Tmin')
    
    coef0,coef1,coef2,coef3,coef4,coef5,coef6 = NASA[0].find('floatArray').text.split()
    vals_to_insert = [specie_name,tmin,tmax,coef0,coef1,coef2,coef3,coef4,coef5,coef6]
    cursor.execute('''INSERT INTO LOW (species_name, TLOW, THIGH, 
                           COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)
    #HIGH
    tmax = NASA[1].get('Tmax')
    tmin = NASA[1].get('Tmin')
    
    coef0,coef1,coef2,coef3,coef4,coef5,coef6 = NASA[1].find('floatArray').text.split()
    vals_to_insert = [specie_name,tmin,tmax,coef0,coef1,coef2,coef3,coef4,coef5,coef6]
    cursor.execute('''INSERT INTO HIGH (species_name, TLOW, THIGH, 
                           COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7) 
                           VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', vals_to_insert)
db.commit()

In [4]:
print("TABLE HIGH")
query = '''SELECT * FROM HIGH'''
high_columns = [col[1] for col in cursor.execute("PRAGMA table_info(HIGH)")]
display(viz_tables(high_columns, query))
print("TABLE LOW")
query = '''SELECT * FROM LOW'''
low_columns = [col[1] for col in cursor.execute("PRAGMA table_info(LOW)")]
display(viz_tables(low_columns, query))

TABLE HIGH


,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,H,1000,5000,"2.50000000E+00,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","2.54716270E+04,",-0.460118
1,O,1000,5000,"2.54205966E+00,","-2.75506191E-05,","-3.10280335E-09,","4.55106742E-12,","-4.36805150E-16,","2.92308027E+04,",4.920308
2,OH,1000,6000,"2.86472886E+00,","1.05650448E-03,","-2.59082758E-07,","3.05218674E-11,","-1.33195876E-15,","3.68362875E+03,",5.701641
3,H2,1000,5000,"2.99142337E+00,","7.00064411E-04,","-5.63382869E-08,","-9.23157818E-12,","1.58275179E-15,","-8.35033997E+02,",-1.355110
4,O2,1000,5000,"3.69757819E+00,","6.13519689E-04,","-1.25884199E-07,","1.77528148E-11,","-1.13643531E-15,","-1.23393018E+03,",3.189166
5,H2O,1000,5000,"2.67214561E+00,","3.05629289E-03,","-8.73026011E-07,","1.20099639E-10,","-6.39161787E-15,","-2.98992090E+04,",6.862817
6,HO2,1000,3500,"4.01721090E+00,","2.23982013E-03,","-6.33658150E-07,","1.14246370E-10,","-1.07908535E-14,","1.11856713E+02,",3.785102
7,H2O2,1000,5000,"4.57316685E+00,","4.33613639E-03,","-1.47468882E-06,","2.34890357E-10,","-1.43165356E-14,","-1.80069609E+04,",0.501137
8,N2,1000,5000,"0.02926640E+02,","0.01487977E-01,","-0.05684761E-05,","0.01009704E-08,","-0.06753351E-13,","-0.09227977E+04,",5.980528
9,Hp,1000,5000,"1.64243522,","2.89759059E-04,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","-4.69510297E+00,",-11.148334


TABLE LOW


,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,H,300,1000,"2.50000000E+00,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","2.54716270E+04,",-0.460118
1,O,300,1000,"2.94642878E+00,","-1.63816649E-03,","2.42103170E-06,,","-1.60284319E-09,","3.89069636E-13,","2.91476445E+04,",2.963995
2,OH,200,1000,"4.12530561E+00,","-3.22544939E-03,","6.52764691E-06,","-5.79853643E-09,","2.06237379E-12,","3.34630913E+03,",-0.690433
3,H2,300,1000,"3.29812431E+00,","8.24944174E-04,","-8.14301529E-07,","-9.47543433E-11,","4.13487224E-13,","-1.01252087E+03,",-3.294094
4,O2,300,1000,"3.21293640E+00,","1.12748635E-03,","-5.75615047E-07,","1.31387723E-09,","-8.76855392E-13,","-1.00524902E+03,",6.034738
5,H2O,300,1000,"3.38684249E+00,","3.47498246E-03,","-6.35469633E-06,","6.96858127E-09,","-2.50658847E-12,","-3.02081133E+04,",2.590233
6,HO2,200,1000,"4.30179801E+00,","-4.74912051E-03,","2.11582891E-05,","-2.42763894E-08,","9.29225124E-12,","2.94808040E+02,",3.716662
7,H2O2,300,1000,"3.38875365E+00,","6.56922581E-03,","-1.48501258E-07,","-4.62580552E-09,","2.47151475E-12,","-1.76631465E+04,",6.785363
8,N2,300,1000,"0.03298677E+02,","0.01408240E-01,","-0.03963222E-04,","0.05641515E-07,","-0.02444855E-10,","-0.01020900E+05,",3.950372
9,Hp,300,1000,"1.64243522,","2.89759059E-04,","0.00000000E+00,","0.00000000E+00,","0.00000000E+00,","-4.69510297E+00,",-11.148334


# Question 2: `WHERE` Statements

1. Write a `Python` function `get_coeffs` that returns an array of 7 coefficients.  
   
   The function should take in two parameters: 1.) `species_name` and 2.) `temp_range`, an indicator variable ('low' or 'high') to indicate whether the coefficients should come from the low or high temperature range.  
   The function should use `SQL` commands and `WHERE` statements on the table you just created in Question 1 (rather than taking data from the XML directly).
```python
def get_coeffs(species_name, temp_range):
    ''' Fill in here'''
    return coeffs
```

2. Write a python function `get_species` that returns all species that have a temperature range above or below a given value. The function should take in two parameters: 1.) `temp` and 2.) `temp_range`, an indicator variable ('low' or 'high').

  When temp_range is 'low', we are looking for species with a temperature range lower than the given temperature, and for a 'high' temp_range, we want species with a temperature range higher than the given temperature.

  This exercise may be useful if different species have different `LOW` and `HIGH` ranges.

  And as before, you should accomplish this through `SQL` queries and where statements.

```python
def get_species(temp, temp_range):
    ''' Fill in here'''
    return coeffs
```

In [5]:
def get_coeffs(species_name,temp_range):
    
            
    if temp_range == 'low':
        #check whether species name is valid
        species_query = '''SELECT DISTINCT SPECIES_NAME FROM LOW'''
        species = cursor.execute(species_query).fetchall()
        clean_species = [i[0].strip(',') for i in species]
        if species_name not in clean_species:
            raise TypeError("Please enter a valid species name")
            
        query = '''SELECT COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7 FROM LOW WHERE SPECIES_NAME = ? '''
        q = cursor.execute(query,(species_name,)).fetchall()
        coeffs=[]
        for i in q[0]:
            if type(i) != float:
                coeffs.append(float(i.strip(",")))
            else:
                coeffs.append(float(i))
    elif temp_range == 'high':
        species_query = '''SELECT DISTINCT SPECIES_NAME FROM HIGH'''
        species = cursor.execute(species_query).fetchall()
        clean_species = [i[0].strip(',') for i in species]
        if species_name not in clean_species:
            raise TypeError("Please enter a valid species name")
            
        query = '''SELECT COEFF_1, COEFF_2, COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7 FROM HIGH WHERE SPECIES_NAME = ? '''
        q = cursor.execute(query,(species_name,)).fetchall()
        coeffs=[]
        for i in q[0]:
            if type(i) != float:
                coeffs.append(float(i.strip(",")))
            else:
                coeffs.append(float(i))
        
    else:
        raise TypeError("Please enter 'high' or 'low' for temp range")
    
    
    return coeffs

In [6]:
get_coeffs('OH','high')

[2.86472886,
 0.00105650448,
 -2.59082758e-07,
 3.05218674e-11,
 -1.33195876e-15,
 3683.62875,
 5.70164073]

In [7]:
get_coeffs('H2','low')

[3.29812431,
 0.000824944174,
 -8.14301529e-07,
 -9.47543433e-11,
 4.13487224e-13,
 -1012.52087,
 -3.29409409]

In [8]:
def get_species(temp, temp_range):
    if temp_range=='low':
        query = '''SELECT SPECIES_NAME FROM LOW WHERE TLOW < ?'''

        q = cursor.execute(query,(temp,)).fetchall()
        coeffs=[]
        for i in q:
            coeffs.append((i[0].strip(",")))

    else:
        query = '''SELECT SPECIES_NAME FROM HIGH WHERE THIGH > ?'''
        q = cursor.execute(query,(temp,)).fetchall()
        coeffs=[]
        for i in q:
            coeffs.append((i[0].strip(",")))
        
    return coeffs

In [9]:
get_species(5000,'high')

['OH']

In [10]:
get_species(200,'low')

[]

# Question 3: `JOIN` STATEMENTS

Create a table named `ALL_TEMPS` that has the following columns:

- `SPECIES_NAME`
- `TEMP_LOW`
- `TEMP_HIGH`

This table should be created by joining the tables `LOW` and `HIGH` on the value `SPECIES_NAME`.

1. Write a `Python` function `get_range` that returns the range of temperatures for a given species_name.

The range should be computed within the `SQL` query (i.e. you should subtract within the `SELECT` statement in the `SQL` query).
```python
def get_range(species_name):
    '''Fill in here'''
    return range
```

Note that `TEMP_LOW` is the lowest temperature in the `LOW` range and `TEMP_HIGH` is the highest temperature in the `HIGH` range.

In [11]:
cursor.execute("DROP TABLE IF EXISTS ALL_TEMPS")
cursor.execute('''
CREATE TABLE ALL_TEMPS AS
    SELECT HIGH.SPECIES_NAME, HIGH.THIGH AS TEMP_HIGH, LOW.TLOW AS TEMP_LOW
    FROM HIGH
        JOIN LOW
            ON HIGH.SPECIES_NAME = LOW.SPECIES_NAME''')
db.commit()

In [12]:
query = '''SELECT * FROM ALL_TEMPS'''
my_columns = [col[1] for col in cursor.execute("PRAGMA table_info(ALL_TEMPS)")]
display(viz_tables(my_columns, query))

,SPECIES_NAME,TEMP_HIGH,TEMP_LOW
0,H,5000,300
1,O,5000,300
2,OH,6000,200
3,H2,5000,300
4,O2,5000,300
5,H2O,5000,300
6,HO2,3500,200
7,H2O2,5000,300
8,N2,5000,300
9,Hp,5000,300


In [13]:
def get_range(species_name):
    query = '''SELECT TEMP_HIGH-TEMP_LOW FROM ALL_TEMPS WHERE SPECIES_NAME = ?'''
    q = cursor.execute(query,(species_name,)).fetchall()
    return q

In [14]:
get_range('H')

[(4700,)]

In [16]:
get_range('OH')

[(5800,)]